In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


data = pd.read_csv('mergerd.csv', encoding='UTF-8-SIG')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

customer_features = data[['CustomerID', 'Region', 'Category', 'TotalValue', 'Quantity']]

first_20_customers = customer_features[customer_features['CustomerID'].isin(
    ["C0001", "C0002", "C0003", "C0004", "C0005", "C0006", "C0007", "C0008", "C0009", "C0010",
     "C0011", "C0012", "C0013", "C0014", "C0015", "C0016", "C0017", "C0018", "C0019", "C0020"]
)]


feature_matrix = first_20_customers[['TotalValue', 'Quantity']]


similarity_matrix = cosine_similarity(feature_matrix)


lookalike_map = {}

for idx, customer_id in enumerate(first_20_customers['CustomerID']):

    similarity_scores = list(enumerate(similarity_matrix[idx]))

    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    top_3 = [(first_20_customers.iloc[i]['CustomerID'], score) for i, score in sorted_scores[1:4]]
    lookalike_map[customer_id] = top_3

import csv

with open('Lookalike.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['CustomerID', 'Lookalikes'])
    for cust_id, lookalikes in lookalike_map.items():
        writer.writerow([cust_id, lookalikes])

print("Lookalike.csv has been created with the top 3 similar customers for the first 20 customers.")

Lookalike.csv has been created with the top 3 similar customers for the first 20 customers.


In [ ]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(feature_matrix)


improved_similarity_matrix = cosine_similarity(normalized_features)


improved_lookalike_map = {}
for idx, customer_id in enumerate(first_20_customers['CustomerID']):

    similarity_scores = list(enumerate(improved_similarity_matrix[idx]))

    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    top_3 = [(first_20_customers.iloc[i]['CustomerID'], score) for i, score in sorted_scores[1:4]]
    improved_lookalike_map[customer_id] = top_3


with open('Improved_Lookalike.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['CustomerID', 'Lookalikes'])
    for cust_id, lookalikes in improved_lookalike_map.items():
        writer.writerow([cust_id, lookalikes])

print("Improved_Lookalike.csv has been created with the top 3 similar customers for all customers.")

Improved_Lookalike.csv has been created with the top 3 similar customers for all customers.
